## Texas Covid19 Predictions - July 7th - August 8th

Megan Riley


In [ ]:
install.packages("forecast")
install.packages("tswge")



also installing the dependencies ‘fracdiff’, ‘lmtest’, ‘urca’, ‘RcppArmadillo’



In [4]:
remove(list = ls())
library(tidyverse)
library(dplyr)
library(tseries)
library(forecast)
library(tswge)


Registered S3 method overwritten by 'xts':
  method     from
  as.zoo.xts zoo 
Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 


ERROR: Error in library(forecast): there is no package called ‘forecast’


In [3]:
#Loading the test and original data

corona_df = read.csv("")
print("Columns in Training:")
names(corona_df)

test_df = read.csv("")
print("Dimensions to predict")
dim(test_df)
print("Columns to predict:")
names(test_df)

[1] "State"                 "Date"                  "Daily_New_Cases"      
 [4] "Curve_Day"             "Pop_Pct"               "Pop_Pct_M1"           
 [7] "Pop_Pct_M2"            "Pop_Pct_M3"            "Pop_Pct_M4"           
[10] "Pop_Pct_M5"            "Pop_Pct_M6"            "Pop_Pct_M7"           
[13] "Three_Day_Avg_Pop_Pct" "Seven_Day_Avg_Pop_Pct" "Pct_Change"           
[16] "Pct_Change_M1"         "Pct_Change_M2"         "Pct_Change_M3"        
[19] "Pct_Change_M4"         "Pct_Change_M5"         "Pct_Change_M6"        
[22] "Pct_Change_M7"         "Three_Day_Avg_Pct_Chg" "Seven_Day_Avg_Pct_Chg"

In [ ]:
texas_corona = corona_df %>% filter(State == "TX")
dim(texas_corona)